<a href="https://colab.research.google.com/github/willjhliang/traffic-sign-recognition/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download dataset from github repo
!rm -r sample_data
!git clone https://github.com/willjhliang/traffic-sign-recognition.git
!mv traffic-sign-recognition/data .
!rm -r traffic-sign-recognition

In [ ]:
import os
import itertools
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt

from sklearn.neighbors import KNeighborsClassifier

In [ ]:
K = 58  # Number of classes

# Data Exploration

In [ ]:
labels = pd.read_csv("data/labels.csv")

train_data = {}
datapath = 'data/images/train'
for k in range(K):
    train_data[k] = []
for f in os.listdir(datapath):
    k = int(f[:3])
    train_data[k].append(Image.open(os.path.join(datapath, f)))


In [ ]:
fig, axs = plt.subplots(6, 10)
fig.set_figheight(15)
fig.set_figwidth(15)
for k, (i, j) in itertools.zip_longest(range(K), list(itertools.product(range(6), range(10))), fillvalue=-1):
    axs[i,j].axis('off')
    if k >= 0:
        axs[i,j].imshow(train_data[k][0])


In [ ]:
img_dist = plt.bar(list(range(K)), [len(train_data[k]) for k in range(K)])

# Models

## Baseline KNN